In [2]:
!pip3 install tensorflow_text>=2.0.0rc0
!pip3 install langchain
!pip3 install gpt-index

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 KB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 KB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.3 MB/s eta 0:00:00
  Create

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_APIKEY"

In [4]:
from langchain.embeddings import TensorflowHubEmbeddings
from gpt_index import LangchainEmbedding

# 埋め込みモデルの準備
embed_model = LangchainEmbedding(TensorflowHubEmbeddings())

In [5]:
from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader, PromptHelper

# インデックスの作成
documents = SimpleDirectoryReader('data').load_data()

In [6]:
# インデックスの生成
index = GPTSimpleVectorIndex(
    documents=documents,
    prompt_helper=PromptHelper(
        max_input_size=4000,  # LLM入力の最大トークン数
        num_output=256,  # LLM出力のトークン数
        chunk_size_limit=500,  # チャンクのトークン数
        max_chunk_overlap=0,  # チャンクオーバーラップの最大トークン数
        separator="。"  # セパレータ
    ),
    embed_model=embed_model,  # 埋め込みモデル
    verbose=True
)

> Adding chunk: あらすじ
結束バンド
後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代に...
> Adding chunk: ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独...
> Adding chunk: きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。

文化祭ライ...
> Adding chunk: その演奏をみんなから拍手喝采されるひとりだったが、コミュ症のひとりはその晴れ舞台に混乱し、観客...
> Adding chunk: Harry Potter is a series of seven fantasy novel...
> [build_index_from_documents] Total LLM token usage: 0 tokens
> [build_index_from_documents] Total embedding token usage: 2007 tokens


In [ ]:
print(index.query(
    "後藤ひとりはどういう性格？", 
    verbose=True
))

> Top 1 nodes:
> [Node 1115eb9a-5210-47c8-95b3-83ef87d95850] [Similarity score:                     0.165062] あらすじ
結束バンド
後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める...
> Searching in chunk: あらすじ
結束バンド
後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代に...
> Initial response: 
後藤ひとりは陰キャで、友達を作ることができず、一人で過ごしていたということから、人見知りで、臆病な性格であると考えられます。
> [query] Total LLM token usage: 611 tokens
> [query] Total embedding token usage: 22 tokens

後藤ひとりは陰キャで、友達を作ることができず、一人で過ごしていたということから、人見知りで、臆病な性格であると考えられます。


In [ ]:
print(index.query(
    "ハリーの性格は？", 
    verbose=True
))

> Top 1 nodes:
> [Node 43d5fefb-4c84-41ef-9dfc-4078ec1ba7a6] [Similarity score:                     0.251621] Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The nov...
> Searching in chunk: Harry Potter is a series of seven fantasy novel...
> Initial response: 
Harry Potter is portrayed as a brave, loyal, and determined character. He is willing to put himself in danger to protect his friends and family, and is often willing to take risks to achieve his goals. He is also shown to be kind and compassionate, and is willing to forgive those who have wronged him.
> [query] Total LLM token usage: 341 tokens
> [query] Total embedding token usage: 12 tokens

Harry Potter is portrayed as a brave, loyal, and determined character. He is willing to put himself in danger to protect his friends and family, and is often willing to take risks to achieve his goals. He is also shown to be kind and compassionate, and is willing to forgive those who have wronged 

In [ ]:
print(index.query(
    "who is the guitar hero？", 
    verbose=True,
    similarity_top_k=2
))

> Top 2 nodes:
> [Node 1115eb9a-5210-47c8-95b3-83ef87d95850] [Similarity score:                     0.308988] あらすじ
結束バンド
後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める...
> [Node c71e5427-7411-4c95-b974-f3a390d0663f] [Similarity score:                     0.2147] その演奏をみんなから拍手喝采されるひとりだったが、コミュ症のひとりはその晴れ舞台に混乱し、観客席にダイブするという暴挙に出てしまう。ひとまずライブは成功を収めるものの、ひとりの奇行により、結束バ...
> Searching in chunk: あらすじ
結束バンド
後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代に...
> Searching in chunk: その演奏をみんなから拍手喝采されるひとりだったが、コミュ症のひとりはその晴れ舞台に混乱し、観客...
> Initial response: 
後藤ひとりがギターヒーローである。彼は中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始めた。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていた。
> Refine context: その演奏をみんなから拍手喝采されるひとりだったが、コミュ症のひとりはその晴れ舞台に混乱し、観客...
> Refined response: 
後藤ひとりがギターヒーローである。彼は中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始めた。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていた。そして、10代限定のロックフェス「未確認ライオット」に参加し、グランプリを目指すことを決めるなど、ひとりのギターヒーローとしての活躍が
> [quer

In [8]:
print(index.query(
    "ハリーの学校では、何を教えている？日本語で答えて", 
    verbose=True,
))

> Top 1 nodes:
> [Node 79824954-b8bf-46f3-b801-985a5f9822f5] [Similarity score:                     0.325184] Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The nov...
> Searching in chunk: Harry Potter is a series of seven fantasy novel...
> Initial response: 
ハリーの学校では、魔法と魔法使いの学問を教えています。
> [query] Total LLM token usage: 338 tokens
> [query] Total embedding token usage: 36 tokens

ハリーの学校では、魔法と魔法使いの学問を教えています。
